# Orchestrating the Cloud with Kubernetes

In [ ]:
gcloud auth list
gcloud config list project

gcloud config set compute/zone us-central1-b
gcloud container clusters create io
gcloud container clusters get-credentials io

### Task 1. Get the sample code

In [ ]:
gsutil cp -r gs://spls/gsp021/* .
cd orchestrate-with-kubernetes/kubernetes

### Task 2. Quick Kubernetes Demo

In [ ]:
kubectl create deployment nginx --image=nginx:1.10.0
kubectl get pods
kubectl expose deployment nginx --port 80 --type LoadBalancer
kubectl get services
curl http://<External IP>:80

### Task 4. Creating Pods

In [ ]:
%%writefile pods/monolith.yaml
apiVersion: v1
kind: Pod
metadata:
  name: monolith
  labels:
    app: monolith
spec:
  containers:
    - name: monolith
      image: kelseyhightower/monolith:1.0.0
      args:
        - "-http=0.0.0.0:80"
        - "-health=0.0.0.0:81"
        - "-secret=secret"
      ports:
        - name: http
          containerPort: 80
        - name: health
          containerPort: 81
      resources:
        limits:
          cpu: 0.2
          memory: "10Mi"

In [ ]:
kubectl create -f pods/monolith.yaml
kubectl get pods
kubectl describe pods monolith

### Task 5. Interacting with pods

In [ ]:
kubectl port-forward monolith 10080:80
curl http://127.0.0.1:10080
curl http://127.0.0.1:10080/secure
curl -u user http://127.0.0.1:10080/login

TOKEN=$(curl http://127.0.0.1:10080/login -u user|jq -r '.token')
curl -H "Authorization: Bearer $TOKEN" 
http://127.0.0.1:10080/secure

kubectl logs monolith
kubectl logs -f monolith

curl http://127.0.0.1:10080

kubectl exec monolith --stdin --tty -c monolith -- /bin/sh

ping -c 3 google.com

### Task 7. Creating a services

In [ ]:
kubectl create secret generic tls-certs --from-file tls/
kubectl create configmap nginx-proxy-conf --from-file nginx/proxy.conf
kubectl create -f pods/secure-monolith.yaml

In [ ]:
%%writefile services/monolith.yaml
kind: Service
apiVersion: v1
metadata:
  name: "monolith"
spec:
  selector:
    app: "monolith"
    secure: "enabled"
  ports:
    - protocol: "TCP"
      port: 443
      targetPort: 443
      nodePort: 31000
  type: NodePort

In [ ]:
kubectl create -f services/monolith.yaml

gcloud compute firewall-rules create allow-monolith-nodeport \
  --allow=tcp:31000

gcloud compute instances list

kubectl get services monolith
kubectl describe services monolith

curl -k https://<EXTERNAL_IP>:31000

### Task 8. Adding labels to pods

In [ ]:
kubectl get pods -l "app=monolith"

kubectl get pods -l "app=monolith,secure=enabled"

kubectl label pods secure-monolith 'secure=enabled'
kubectl get pods secure-monolith --show-labels

kubectl describe services monolith | grep Endpoints

gcloud compute instances list
curl -k https://<EXTERNAL_IP>:31000

### Task 10. Creating deployments

In [ ]:
%%writefile deployments/auth.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: auth
spec:
  selector:
    matchlabels:
      app: auth
  replicas: 1
  template:
    metadata:
      labels:
        app: auth
        track: stable
    spec:
      containers:
        - name: auth
          image: "kelseyhightower/auth:2.0.0"
          ports:
            - name: http
              containerPort: 80
            - name: health
              containerPort: 81

In [ ]:
kubectl create -f deployments/auth.yaml
kubectl create -f services/auth.yaml

kubectl create -f deployments/hello.yaml
kubectl create -f services/hello.yaml

kubectl create configmap nginx-frontend-conf --from-file=nginx/frontend.conf
kubectl create -f deployments/frontend.yaml
kubectl create -f services/frontend.yaml

kubectl get services frontend
curl -k https://<EXTERNAL-IP>